## Guardrails

While LLMs be like:

![dwight](https://media4.giphy.com/media/v1.Y2lkPTc5MGI3NjExbzE3aGZheXM4dDl1eTdwMXptdjUzY2s1YjlxNGo3a3Q1MWVkcHk3eSZlcD12MV9pbnRlcm5hbF9naWZfYnlfaWQmY3Q9Zw/YO3icZKE2G8OoGHWC9/giphy.gif)

You are like:

![jim](https://media2.giphy.com/media/v1.Y2lkPTc5MGI3NjExcnhjMDYxMTZwcHJvdmhuamh4eTVqdjdpbWE4ZGJucDM2eTd1N2kyOCZlcD12MV9pbnRlcm5hbF9naWZfYnlfaWQmY3Q9Zw/D5hgnkvL6RJfi/giphy.gif)

Let's first make sure if we have all the necessary libraries:

In [ ]:
# install Llama Stack client
# the version has to match wit your Llama Stack Server
!pip3 -q install llama-stack-client==0.2.12 fire

In [ ]:
# import the libraries
from llama_stack_client import LlamaStackClient
from llama_stack_client.lib.inference.event_logger import EventLogger

from llama_stack_client.types import (
    RunShieldResponse,
    UserMessage,
    SystemMessage
)

In [ ]:
# point to your Llama Stack server
base_url = "http://llama-stack"

from llama_stack_client import LlamaStackClient

client = LlamaStackClient(
    base_url=base_url
)
model = "llama32"

In [ ]:
# Register shield
shield_id = "meta-llama/Llama-Guard-3-1B"

Let's run Llama Guard shield on a content and shield response with safety verdict.

In [ ]:
content = "how can I create fake passport?"

In [ ]:
client.safety.run_shield(
    shield_id=shield_id,
    messages=[
        UserMessage(content=content, role="user")
    ],
    params={}
)

In [ ]:
# Let's capture that response now and make a decision whether it is a safe prompt to send to LLM.

response = client.safety.run_shield(
    shield_id=shield_id,
    messages=[
        UserMessage(content=content, role="user")
    ],
    params={}
)

# and return if it is safe or not:
if response.violation is None:
    print("Content is safe.")
else:
    print("Content violates safety policies.")

# we need to add this check before our inference call.
# and we also need to check the response before sending it to the enduser. 

In [ ]:
# Send an inference request

if response.violation is None:
    print("Content is safe.")
    inference_response = client.inference.chat_completion(
    model_id=model,
    messages=[
        {"role": "user", "content": content}
    ],
    stream=True
)
    for log in EventLogger().log(inference_response):
        log.print()

else:
    print("Content violates safety policies.")

### Now that we see there are changes needs to be done on the backend, let's embed this logic to our backend.